In [103]:
import json
import re

data = json.loads(open("data/data.json", encoding="utf-8").read())

In [104]:
import attrs
import typing as t


@attrs.define(on_setattr=attrs.setters.validate)
class CarListing:
    id: str
    url: str
    tytul: str
    opis: str
    cena: int
    prywatne: bool
    model: str | None
    rok_produkcji: int | None
    paliwo: t.Literal["Diesel", "Benzyna"]
    przebieg: int | None
    pojemnosc: int | None
    skrzynia_biegow: t.Literal["Automatyczna", "Manualna"]
    kraj_pochodzenia: str | None
    moc: int | None

In [107]:

def parse(data_dict: dict):
    parsed_dict = {
        "id": data_dict["id"].split(" ")[1],
        "url": data_dict["url"],
        "tytul": data_dict["title"],
        "opis": re.sub(r'\n+', '.', data_dict["description"]),  # TODO probably more cleaning
        "cena": int(data_dict["price"].split("zł")[0].replace(" ", "")),
        "prywatne": data_dict["parameters"].pop(0) == "Prywatne" if len("parameters") else None
    }
    
    attributes_dict = {attribute.split(":")[0].strip(): attribute.split(":")[1].strip() for attribute in data_dict["parameters"]}
    
    parsed_dict = {
        **parsed_dict,
        "model": attributes_dict.get("Model", None),
        "rok_produkcji": attributes_dict.get("Rok produkcji", None),
        "paliwo": attributes_dict.get("Paliwo", None),
        "przebieg": int(attributes_dict["Przebieg"].split("km")[0].replace(" ", "")) if "Przebieg" in attributes_dict else None,
        "pojemnosc": int(attributes_dict["Poj. silnika"].split("cm")[0].replace(" ", "")) if "Poj. silnika" in attributes_dict else None,
        "skrzynia_biegow": attributes_dict.get("Skrzynia biegów", None),
        "kraj_pochodzenia": attributes_dict.get("Kraj pochodzenia", None),
        "moc": int(attributes_dict["Moc silnika"].split(" ")[0]) if "Moc silnika" in attributes_dict else None,
    }

    return CarListing(**parsed_dict)

In [109]:
res = []

for entry in data:
    res.append(parse(entry))


In [110]:
res[0]

CarListing(id='1031072842', url='https://www.olx.pl/d/oferta/bmw-e36-325i-192km-r6-m54b25-daily-drift-szpera-hydro-mpakiet-barany-f-CID5-ID17Mh98.html', tytul='BMW e36 325i 192km R6 M54b25 daily/drift szpera hydro mpakiet barany f', opis='Witam.Sprzedam moją maskotkę i najlepszego kamrata BMW e36 318i/325i.Dużo by tu pisać ale w skrócie:.Auto zbudowane pod daily/drift czyli środek cywilny z działającym radiem i wkręcanym hydro:.- silnik R6 2.5 192km M54b25 z e46, przed włożeniem sprawdzony,(wyjety z jeżdżącego auta) wyczyszczony, uszczelniony(nowe uszczelki).-filtr paliwa przeniesiony wyżej z NOWYMI przewodami paliwowymi, można śmiało trzeć podłużnicami, brak możliwości jego zerwania.-do wiązki wpięte OBD (można podpiąć kompa i połączyć się z silnikiem).-NOWE świece.-NOWE kolektory barany schmiedmann.-komora silnika wyczyszczona i pomalowana w róż.-M pakiet zderzak przód , zderzak tył, listwy progowe, szerokie listwy.-GT Class 2.-duże hamulce z46 Mpak wypiaskowane, pomalowane proszkiem